In [14]:
import time
import copy
import random

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans

from graph_tree_list_module import *


In [15]:
raw = pd.read_csv(r'../Data/train.csv')
# print(raw)

random_seed = 42

# print(list(raw.columns)[:])

In [16]:
data = raw.iloc[:,2:-1]
data = data.fillna(0)

data = data[:2000]
print(data.shape)
scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(data)

# label
y = raw.iloc[:,-1]
# print(y)

col_name = list(raw.columns)[2:-1]
print(col_name)
# print(normalized_data)

(2000, 14)
['Popularity', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_in min/ms', 'time_signature']


In [17]:
data_min = scaler.data_min_
data_max = scaler.data_max_

# print(data_min, data_max)

In [18]:
def test_query_generator():
    return [random.uniform(data_min[i], data_max[i]) for i in range(len(data_max))]

In [19]:
print(test_query_generator())

[np.float64(77.6201413878787), np.float64(0.4090567559969476), np.float64(0.575423364847147), np.float64(6.7565859243622794), np.float64(-27.153600234097784), np.float64(0.5895852713104587), np.float64(0.5437055345736003), np.float64(0.20732910237381927), np.float64(0.4264395607462939), np.float64(0.4202558740870492), np.float64(0.9502616135313517), np.float64(152.062667318954), np.float64(366088.9029069345), np.float64(1.03116035445667)]


In [20]:
import time

class Timer:
    def __enter__(self):
        self.start_time = time.perf_counter()
        return self

    def __exit__(self, *args):
        self.end_time = time.perf_counter()
        self.elapsed_time = self.end_time - self.start_time
        print(f"Elapsed time: {self.elapsed_time} seconds")

In [21]:
test_tree = graph_tree(attr_name = col_name, X = normalized_data, X_maxmin=[scaler.data_max_,scaler.data_min_])
# test_tree.show_root()
with Timer():
    test_tree.build_tree(min_threshold_for_clustering=50)

cur node 0
optimal K 2
cur node 1
optimal K 6
cur node 2
optimal K 6
cur node 3
optimal K 2
cur node 4
optimal K 2
cur node 5
optimal K 3
cur node 6
cur node 7
optimal K 2
cur node 8
optimal K 9
cur node 9
optimal K 2
cur node 10
optimal K 2
cur node 11
optimal K 2
cur node 12
optimal K 2
cur node 13
optimal K 4
cur node 14
optimal K 2
cur node 15
optimal K 4
cur node 16
optimal K 10
cur node 17
optimal K 2
cur node 18
optimal K 2
cur node 19
cur node 20
cur node 21
cur node 22
cur node 23
optimal K 2
cur node 24
cur node 25
cur node 26
cur node 27
cur node 28
cur node 29
cur node 30
cur node 31
cur node 32
cur node 33
optimal K 2
cur node 34
optimal K 2
cur node 35
optimal K 2
cur node 36
optimal K 2
cur node 37
optimal K 2
cur node 38
optimal K 10
cur node 39
optimal K 2
cur node 40
optimal K 2
cur node 41
cur node 42
cur node 43
cur node 44
cur node 45
optimal K 2
cur node 46
optimal K 2
cur node 47
cur node 48
cur node 49
cur node 50
cur node 51
cur node 52
cur node 53
cur node 54


In [22]:
# import pickle

# with open("graph_tree.pkl", 'wb') as f:
#     pickle.dump(test_tree, f)

In [23]:
seaching = test_query_generator()
with Timer():
    cur = test_tree.search(seaching)
print(cur.id)

Elapsed time: 0.0017816000035963953 seconds
60


In [24]:
foo = 204
with Timer():
    res = test_tree.discovery_search(test_tree.nodes[foo].center, test_tree.nodes[foo], lv_degree=2)
print(res.keys())
print(res["sorted_cos_sim"])
print(res["euclide_dist"])
print([n.id for n in res["sorted_nodes"]])


Elapsed time: 3.7999998312443495e-06 seconds


IndexError: list index out of range

In [12]:
a = test_tree.get_midle_graph_data(19, lv_degree=3)
print(a)
print([n.id for n in a["sorted_nodes"]])

reflection_node: 0 at lv up: 3
generate cos sim, ecul dist, and nodes sorted by cos sim
{'sorted_cos_sim': [np.float64(-0.8413730479488191), np.float64(-0.8034750247421106), np.float64(-0.6443578450198645), np.float64(-0.5359142835620883), np.float64(-0.46513974041933465), np.float64(-0.4312990219958266), np.float64(-0.2655401713695995), np.float64(-0.0423949413639789), np.float64(0.10800422052851304), np.float64(0.13626615375362464), np.float64(0.192569476915936), np.float64(0.2795794855968571), np.float64(0.28323786086515), np.float64(0.33027038157517796), np.float64(0.34411118153487774), np.float64(0.4233136247173815), np.float64(0.4474326732605564), np.float64(0.45623098751593094), np.float64(0.5251581960227801), np.float64(0.5710393511753408), np.float64(0.5721063214508092), np.float64(0.6652485701488396), np.float64(0.6691706034939141), np.float64(0.6805262815513853), np.float64(0.6963692560122966), np.float64(0.796658587425673), np.float64(0.8175335550471369), np.float64(1.0)], 

In [25]:
test_tree.print_tree(test_tree.nodes[0])

└── 0
    ├── 1
    │   ├── 3
    │   │   ├── 15
    │   │   │   ├── 47
    │   │   │   ├── 48
    │   │   │   ├── 49
    │   │   │   └── 50
    │   │   └── 16
    │   │       ├── 51
    │   │       ├── 52
    │   │       ├── 53
    │   │       ├── 54
    │   │       ├── 55
    │   │       ├── 56
    │   │       ├── 57
    │   │       ├── 58
    │   │       ├── 59
    │   │       └── 60
    │   ├── 4
    │   │   ├── 17
    │   │   │   ├── 61
    │   │   │   └── 62
    │   │   └── 18
    │   │       ├── 63
    │   │       │   ├── 95
    │   │       │   └── 96
    │   │       └── 64
    │   ├── 5
    │   │   ├── 19
    │   │   ├── 20
    │   │   └── 21
    │   ├── 6
    │   ├── 7
    │   │   ├── 22
    │   │   └── 23
    │   │       ├── 65
    │   │       │   ├── 97
    │   │       │   └── 98
    │   │       └── 66
    │   └── 8
    │       ├── 24
    │       ├── 25
    │       ├── 26
    │       ├── 27
    │       ├── 28
    │       ├── 29
    │       ├── 30
    │       ├── 31
    │    

In [14]:
from collections import Counter

"""
create a new index (something like tf-idf)
data reduction rate
vs
accurate
"""
def score(search_data, acc, smoothing = 0.001, lam = 0.5):
    return lam*(acc/100)+(1-lam)*(1-(search_data/(1000+smoothing)))

my_score = []
normal_kmeans_score = []

tmp = [i*100 for i in range(10)]
for i in tmp:
    b = i
    u = i+100
    # print(normalized_data[b:u])
    print(f"-- mine result{i}")
    seaching = np.mean(normalized_data[b:u], axis=0)
    result = test_tree.search(seaching)
    
    print("search number: ", len(result.index))
    right = np.where((result.index>=b) & (result.index<u))[0] # return index
    print("hitted right result:", len(right))
    my_score.append(score(len(result.index), len(right)))

    ############
    print("-- kmeans result")
    checking = KMeans(n_clusters=10, random_state=0).fit(normalized_data)

    counts1 = Counter(checking.labels_)
    print(counts1)

    counts2 = Counter(checking.labels_[b:u])
    print(counts2)
    belongs_group = counts2.most_common()[0][0]
    print("belongs to group: ", belongs_group, "\n")
    # print(counts1.get(belongs_group), counts2.get(belongs_group))
    normal_kmeans_score.append(score(counts1.get(belongs_group), counts2.get(belongs_group)))

    

-- mine result0
search number:  5
hitted right result: 0
-- kmeans result
Counter({np.int32(9): 944, np.int32(0): 659, np.int32(3): 656, np.int32(1): 619, np.int32(4): 607, np.int32(7): 577, np.int32(5): 299, np.int32(8): 281, np.int32(6): 191, np.int32(2): 167})
Counter({np.int32(9): 22, np.int32(1): 17, np.int32(0): 13, np.int32(4): 12, np.int32(7): 10, np.int32(3): 9, np.int32(8): 6, np.int32(5): 5, np.int32(2): 5, np.int32(6): 1})
belongs to group:  9 

-- mine result100
search number:  5
hitted right result: 0
-- kmeans result
Counter({np.int32(9): 944, np.int32(0): 659, np.int32(3): 656, np.int32(1): 619, np.int32(4): 607, np.int32(7): 577, np.int32(5): 299, np.int32(8): 281, np.int32(6): 191, np.int32(2): 167})
Counter({np.int32(3): 17, np.int32(9): 15, np.int32(4): 15, np.int32(7): 11, np.int32(5): 10, np.int32(1): 10, np.int32(0): 9, np.int32(6): 6, np.int32(8): 5, np.int32(2): 2})
belongs to group:  3 

-- mine result200
search number:  5
hitted right result: 0
-- kmeans resu

In [15]:
print(my_score)
print(normal_kmeans_score)

print(sum(my_score))
print(sum(normal_kmeans_score))

[0.4975000024999975, 0.4975000024999975, 0.4975000024999975, 0.4975000024999975, 0.4975000024999975, 0.4975000024999975, 0.4975000024999975, 0.4975000024999975, 0.4975000024999975, 0.4995000004999995]
[0.13800047199952797, 0.25700032799967204, 0.13300047199952797, 0.13300047199952797, 0.27550032949967046, 0.12300047199952799, 0.2965002884997115, 0.28650030349969646, 0.12300047199952799, 0.14300047199952798]
4.9770000229999765
1.9085040814959182
